# Categories  in NL wikipedia

In deze opdracht gaan we een gestructureerde kennisbank maken gebaseerd op de NL wikipedia.

We doen 2 dingen

1. We maken het netwerk (het is geen boom maar een DAG) van categorieen.
2. Voor elke pagina, geven we aan  onder welke categorieen het _rechtstreeks_ valt
     

### Categorie-informatie
* Voor elke pagina kunnen we de verzameling categorieen die rechtstreeks bij die pagina horen (die dus onder aan de pagina staan) vergelijken met de categorieen die we ook met beghulp van overerving zouden krijgen.
    * Staat er bijvoorbeeld vaak redundante informatie op paginas? 
    * Hoe verhouden het aantal  expliciete en impliciete categorieen zich?
    
 

## Categories



### Doel
* We willen een netwerk van categori&euml;en distelleren uit de NL wikipedia.
* Dat bestaat en kunnen we raadplegen via <https://nl.wikipedia.org/wiki/Speciaal:Categorieboom>
* Nu willen we die data zelf hebben en in een kennisbank opslaan.

#### Hoe kunnen we dat doen?

1. Probeer de structuur van de categorieen ergens binnen Wikipedia te vinden.
2. Hier is iemand die hetzelfde wilde: <http://stackoverflow.com/questions/12227134/how-to-fetch-category-tree-of-wiki>

### Dumps/data
* Dumps: <https://dumps.wikimedia.org/nlwiki/20150602/>
* API: <https://www.mediawiki.org/wiki/API%3aCategorymembers>
* Je bent niet de eerste met deze opdracht ....
    * Zoek op het web.
    * Bijvoorbeeld: <http://stackoverflow.com/questions/8088226/content-of-infobox-of-wikipedia/21107068#21107068>
    * Zie ook <https://www.mediawiki.org/wiki/Alternative_parsers> voor nog meer mediawiki parsers




# Opdrachten

## Categorieen DAG

### Download en representeer. 30%
1. Verzin een datastructuur voor de categorieen DAG. Haal de NL wikipedia categorie structuur binnen, en sla die op in je datastructuur.

### Analyseer 30%
* Doe dit op de file <http://maartenmarx.nl/teaching/KennisSystemen/Data/categorie_diepte6.csv.gz> 
1. Hoeveel unieke categorieen zijn er?
1. ~~Hoe diep is de DAG? Wat is het langste pad van de root naar een blad, en wat is het kortste?~~
3. Welke categorie heeft de meeste kinderen?
4. Definieer de functie die alle  nakomelingen (descendants) van een gegeven knoop teruggeeft.
5.  compute the average branching factor

## Categorieen en paginas 30%
1. Bepaal voor elke Wikipedia pagina tot welke categorieen die rechtstreeks behoort, en tot welke categorieen die behoort dmv overerving.
    * Kies een slimme manier om dit te representeren.
1. Bepaal voor elke Wikipedia pagina, hoeveel van de expliciet genoemde categorieen, ook via overerving hadden kunnen worden afgeleid.
    * Wat is het gemiddelde per pagina?
    
## Reflecteer 10%
Beschrijf in een aantal punten wat er makkelijk was en wat lastig was aan de opdracht. 
Geef goed weer wat voor technieken en mogelijkheden je hebt gebruikt.


## Bonus 10%
Volgens de literatuur is de wikipedia isa hierarchy niet juist want **het bevat cycles**. Op de file van "diepte 6" is dit misschien ook al het geval. Schrijf een kort programma dat detecteert of er cycles zijn, en die als er een is, ook eentje uitprint.

Geef aan hoe je dit zou kunnen verhelpen.

## Hints

1. Het geeft niet als je code niet de hele Wikipedia aankan. Begin steeds met kleine files/stukjes, anders wacht je heel erg lang. 
2. Voor het downloaden van de Categorie structuur werkte voor mij het recursief aanroepen van de API het beste. **Zie de voorbeeld code hieronder.**
    * Ga niet verder dan diepte 3 of 4 vanaf de root.
    * zorg dat je geen dubel werk doet. De categorie structuur is een DAG, dus je komt via allerlei paden weer op dezelfde knopen uit. 
    * Begin bovenaan, en vraag de kinderen op, en ga dan recursief (breadth-first of depth-first?) met de kinderen hetzelfde doen.
    * zet een limiet op de diepte dmv een tellertje
3. Hoe simpeler je je netwerk representeert, hoe makkelijker de analyse. 
    * Denk aan Prolog: hoe zou je het daar doen?
    * Hoe zou je het in een database doen? Of in een spreadsheet? 
    * Zit daar eigenlijk veel verschil tussen?
4. Gebruik de voorbeeld code om de wikipedia XML dump te downloaden en voor elke pagina de categorien eruit te trekken.
    * Experimenteer met een **heel kleine file**: 10 paginas is meer dan voldoende. 
    * Als ie dan werkt, zet je programma aan op de hele bubs, en ga koffie drinken.
    * Als je uit werkgeheugen loopt, los het dan op door stromend in te lezen, of geef het aan in je code.
5. Gebruik itertools om de union van veel verzamelingen tegelijk te nemen. Bijvoorbeeld:
    * voor   `pak_children(n)` een functie die de verzameling van alle kinderen van `n` teruggeeft, heb je hier de verzameling kleinkinderen van `Ida`.
    * `set(itertools.chain(*[pak_children(n) for n in pak_children(Ida)]))`
6. Om de categorieen uit een pagina te vissen is het gebruik van een reguliere expressie heel erg handig.

## De datastructuur


In [67]:
# Datastructoor voor een Directed Asyclic Graph
# We gebruiken hiervoor een dict. In deze dict definiëren we de child nodes van een node als een dict van dat element.
# De leaf nodes zijn de diepst geneste elementen in de dict.
# Op deze manier kan je eenvoudig child nodes vinden van een node.

# Hier onder een voor
dataTree = {"Uppernode1": {"Node1": {"Leaf1": {}, "Leaf2": {}}, "Node2": {} }, "Uppernode2": {} }
print dataTree

{'Uppernode2': {}, 'Uppernode1': {'Node1': {'Leaf1': {}, 'Leaf2': {}}, 'Node2': {}}}


## De analyse
Voor deze analysevragen hebben we de bijgeleverde .csv file gebruikt

#### In totaal zijn er 13765 categoriën in de CSV file
#### Als we Wikipedia als categorie niet mee tellen is de meest voorkomende categorie Film naar regisseur
#### De functie twee blokken hier onder print alle nakomelingen als een dict
#### De gemiddelde branching facor is 6.6, zoals te zien in de berekening hier onder

In [108]:
# Vragen over categoriën

import csv

dataset = dict()
with open('categorie_diepte6.csv','rb') as csvfile:
    dataReader = csv.reader(csvfile, delimiter='\t')
    for row in dataReader:
        if row[1] in dataset:
            dataset[row[1]] += 1
        else:
            dataset[row[1]] = 1
            
    print "Er zijn", len(list(dataset)), "categoriën"
    mostChildren = list(dataset)[0]
    
    childNodeSum = 0.0
    for node in dataset:
        childNodeSum += dataset[node]
        if dataset[node] > dataset[mostChildren] and not node == "Wikipedia":
            mostChildren = node
    print "De categorie met de meeste kinderen is", mostChildren
    branchingFactor = childNodeSum / len(list(dataset))
    print "De branching factor is", branchingFactor

 Er zijn 13765 categoriën
De categorie met de meeste kinderen is Film naar regisseur
De branching factor is 6.5949872866


In [104]:
# Nakomelingen functie
def children(dataSet, key):
    childrenDict = dict()
    nodeFound = False
    for node in dataSet:
        if node == key:
            childrenDict = dataSet[node]
            nodeFound = True
            return childrenDict
    if not nodeFound:
        for node in dataSet:
            temp = children(dataSet[node],key)
            if temp:
                nodeFound = True
                return temp
    if nodeFound:
        return childrenDict
    else:
        return {}


dataTree = {"Uppernode1": {"Node1": {"Leaf1": {}, "Leaf2": {}}, "Node2": {} }, "Uppernode2": {} }
childrenList = children(dataTree, "Node1")
print childrenList


{'Leaf1': {}, 'Leaf2': {}}


## Reflectie
Het bedenken en opzetten van de datatructuur was niet heel ingewikkeld. Het was daarna wel even puzzelen om een handige functie te maken die alle nakomelingen van een categorie kon geven. Het rekenen met en analyseren van de data, als het een maal in de dict zat was goed te doen.

Voor de wikipedia:




# Categorieen en pagina's

In [78]:
import requests  # http://docs.python-requests.org/en/latest/
import json
import itertools
import warnings
import sys
warnings.filterwarnings('ignore')

def subcategories(category):
    url="https://nl.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Categorie:Alles&cmlimit=500&cmtype=subcat&format=json&rawcontinue"
    '''Geef het paar (categorie, lijst van subcategorieen)  terug, gegeven input category. 
    Maximaal 500 (dit stel je in  in  de wikipedia api)'''
    cat_url=url.replace('Categorie:Alles', category)
    try:
          # download the data from the API
          # turn into a dict
        top= requests.get(cat_url) # download the data from the API
        data=top.json()  # turn into a dict
        subs={}
        query = data.get('query')
        catmem = query.get("categorymembers")
        for i in range(0,len(catmem)):
            cat = catmem[i].get('title')
            subs[i] = cat
        output = {category: subs}
    except:
        output={}  # bij een error schrijven we maar een lege lijst terug
        #e = sys.exc_info()[0]
        #write_to_page( "<p>Error: %s</p>" % e )
    return output

# test

parent = 'Categorie:Wetenschap'
categories[0] = subcategories(parent)
cats = categories.get(parent)
for i in range(0, len(cats)):
    parent = cats.get(i)
    newcats = subcategories(parent)
    if newcats:
        categories[i+1] = newcats
        for j in range(0, len(newcats)):
            parent = newcats.get(j)
            if parent:
                categories[j+100] = subcategories(parent)
    
print categories


{0: {'Categorie:Wetenschap': {0: u'Categorie:Wikipedia:Beginnetje wetenschap & technologie', 1: u'Categorie:Wetenschap naar land', 2: u'Categorie:Wetenschapper', 3: u'Categorie:Exacte wetenschap', 4: u'Categorie:Formele wetenschap', 5: u'Categorie:Geesteswetenschappen', 6: u'Categorie:Hoger onderwijs', 7: u'Categorie:Indologie', 8: u'Categorie:Interdiscipline', 9: u'Categorie:Kunstkritiek', 10: u'Categorie:Minister van Wetenschap', 11: u'Categorie:Natuurwetenschappen', 12: u'Categorie:Onderzoek', 13: u'Categorie:Onopgelost probleem', 14: u'Categorie:Sociale wetenschap', 15: u'Categorie:Toegepaste wetenschap', 16: u'Categorie:Wetenschappelijk genootschap', 17: u'Categorie:Wetenschappelijk televisieprogramma', 18: u'Categorie:Wetenschappelijke fraude', 19: u'Categorie:Wetenschappelijke literatuur', 20: u'Categorie:Wetenschappelijke software', 21: u'Categorie:Wetenschappelijke wet', 22: u'Categorie:Wetenschapsfilosofie', 23: u'Categorie:Wetenschapsgeschiedenis', 24: u'Categorie:Wetenschap


# Wat probeersels  (negeer dit gerust)

In [18]:
#!mkdir NLWikipedia
%cd NLWikipedia/

[Errno 2] No such file or directory: 'NLWikipedia/'
/home/kajzzer/Documents/UvA/20162017/Semester 1/Kennissystemen/Python


# Recursief aanroepen van api

In [4]:
import requests  # http://docs.python-requests.org/en/latest/
import json
import itertools
import warnings
import sys
warnings.filterwarnings('ignore')

# get the subcategories of the top wikipedia category "Alles"
url="https://nl.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Categorie:Alles&cmlimit=500&cmtype=subcat&format=json&rawcontinue"

# testje met de top 
top= requests.get(url) # download the data from the API
topdict=top.json()  # turn into a dict
topdict


{u'query': {u'categorymembers': [{u'ns': 14,
    u'pageid': 66068,
    u'title': u'Categorie:Cultuur'},
   {u'ns': 14, u'pageid': 63562, u'title': u'Categorie:Geschiedenis'},
   {u'ns': 14, u'pageid': 133035, u'title': u'Categorie:Heelal'},
   {u'ns': 14, u'pageid': 973784, u'title': u'Categorie:Lijsten'},
   {u'ns': 14, u'pageid': 176685, u'title': u'Categorie:Mens en maatschappij'},
   {u'ns': 14, u'pageid': 68387, u'title': u'Categorie:Natuur'},
   {u'ns': 14, u'pageid': 1677948, u'title': u'Categorie:Persoon'},
   {u'ns': 14, u'pageid': 63481, u'title': u'Categorie:Wetenschap'},
   {u'ns': 14, u'pageid': 67372, u'title': u'Categorie:Wikipedia'}]}}

In [3]:
# Het werkpaard van alles: Maak af

def subcategories(category):
    '''Geef het paar (categorie, lijst van subcategorieen)  terug, gegeven input category. 
    Maximaal 500 (dit stel je in  in  de wikipedia api)'''
    cat_url=url.replace('Categorie:Alles', category)
    try:
          # download the data from the API
          # turn into a dict
        subs=  
    except:
        subs=[]  # bij een error schrijven we maar een lege lijst terug
        #e = sys.exc_info()[0]
        #write_to_page( "<p>Error: %s</p>" % e )
    return category, subs

# test

subcategories('poepoe'), subcategories('Categorie:Techniek')

SyntaxError: invalid syntax (<ipython-input-3-f3ac7f51ece3>, line 10)

## Category tabel

In [10]:
#! wget --no-check-certificate https://dumps.wikimedia.org/nlwiki/20150602/nlwiki-20150602-category.sql.gz
#!gunzip nlwiki-20150602-category.sql.gz
#!head -100 nlwiki-20150602-category.sql
!gzip nlwiki-20150602-category.sql

gzip: can't stat: nlwiki-20150602-category.sql (nlwiki-20150602-category.sql): No such file or directory


## mmm

* Aan bovenstaande file heb je niet veel, want je krijgt de categorie structuur niet! 
* Kijk maar naar het tabelformaat en de voorbeelden:

```
CREATE TABLE `category` (
  `cat_id` int(10) unsigned NOT NULL AUTO_INCREMENT,
  `cat_title` varbinary(255) NOT NULL DEFAULT '',
  `cat_pages` int(11) NOT NULL DEFAULT '0',
  `cat_subcats` int(11) NOT NULL DEFAULT '0',
  `cat_files` int(11) NOT NULL DEFAULT '0',
  PRIMARY KEY (`cat_id`),
  UNIQUE KEY `cat_title` (`cat_title`),
  KEY `cat_pages` (`cat_pages`)
) ENGINE=InnoDB AUTO_INCREMENT=14627193 DEFAULT CHARSET=binary;
/*!40101 SET character_set_client = @saved_cs_client */;

--
-- Dumping data for table `category`
--

/*!40000 ALTER TABLE `category` DISABLE KEYS */;
INSERT INTO `category` VALUES (1,'Plaats_in_Portugal',3577,6,0),(2,'Wikipedia:Afbeelding_gewenst',126,7,0),(3,'Spionageroman',16,2,0),(4,'Ridderorde_in_Polen',16,1,0),(5,'Nederlands_kunstenaar',481,24,0),(6,'Nederlands_kunstschilder',2203,5,0),(7,'Wikipedia:Werk_in_uitvoering',17,0,0),(8,'South_Park',17,2,0)
```

* Als we de tabel vergelijken met bijvoorbeeld <https://nl.wikipedia.org/wiki/Albert_Speer> zien we dat er alleen instaat welke categorien elke pagina bevat.
* We hebben nog niet de boom zelf.

In [4]:
# We kunnen ook alle artikelen ophalen.
# Dit hier is de grootste: er zijn ook veel kleinere. Begin daar liever mee te werken.

!wget --no-check-certificate  https://dumps.wikimedia.org/nlwiki/20160720/nlwiki-20160720-pages-articles1.xml.bz2

--2016-07-28 10:44:48--  https://dumps.wikimedia.org/nlwiki/20160720/nlwiki-20160720-pages-articles1.xml.bz2
Resolving dumps.wikimedia.org... 208.80.154.11, 2620::861:1:208:80:154:11
Connecting to dumps.wikimedia.org|208.80.154.11|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 152370024 (145M) [application/octet-stream]
Saving to: `nlwiki-20160720-pages-articles1.xml.bz2'

100%[======================================>] 152,370,024 1.95M/s   in 74s     

2016-07-28 10:46:03 (1.97 MB/s) - `nlwiki-20160720-pages-articles1.xml.bz2' saved [152370024/152370024]



In [73]:
# This is a huge file, we just want to inspect it now, and so only decompress the first 10.000 lines
# Run this in the shell. From here you get an error
!bzcat "nlwiki-20150602-pages-articles.xml.bz2" |head -10000 > small.xml


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = nlwiki-20150602-pages-articles.xml.bz2, output file = (stdout)


In [5]:
!head -80 NLWikipedia/small.xml



<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="nl">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <dbname>nlwiki</dbname>
    <base>http://nl.wikipedia.org/wiki/Hoofdpagina</base>
    <generator>MediaWiki 1.26wmf7</generator>
    <case>first-letter</case>
    <namespaces>
      <namespace key="-2" case="first-letter">Media</namespace>
      <namespace key="-1" case="first-letter">Speciaal</namespace>
      <namespace key="0" case="first-letter" />
      <namespace key="1" case="first-letter">Overleg</namespace>
      <namespace key="2" case="first-letter">Gebruiker</namespace>
      <namespace key="3" case="first-letter">Overleg gebruiker</namespace>
      <namespace key="4" case="first-letter">Wikipedia</namespace>
      <namespace key="5" case="first-letter">Overleg Wikipedia</nam

### Extracting information with Beautifulsoup

* Dit is een makkelijke Python module om met HTML en XML fies om te gaan.
* Hieronder zie je wat voorbeelden

### Extracting information with lxml
* Dit werkt net wat lastiger maar is wel veel stabieler en kan ook joekels van files aan.
* Je hoeft zelfs een file niet eerst te "ontzippen", maar kan hem stromend in lxml inlezen.
* Dat betekent dat je niet snel geheugen problemen zal hebben: want jij hoeft maar steeds 1 pagina in geheugen te hebben.
* Dit is een deel van mijn antwoord:
* Je ziet dat je rekening moet houden met de namespaces (dat hoeft in Beautifulsoup niet).
* Alles werkt net wat lastiger dan in BeautifulSoup, maar het schaalt wel.

```
from lxml import etree as et
from bz2file import BZ2File

def create_page_category_dict(f):
    with BZ2File(f) as xml_file:
        context = etree.iterparse(xml_file)
        
        pages_categories_dict={} 
        for action, elem in context:
             if elem.tag== '{http://www.mediawiki.org/xml/export-0.10/}page':
                title=  
                pagetext=  
                cats= find_categories(pagetext) # own function which extract the categories from the text of a page using a regular expression
                pages_categories_dict[title]=cats # update the dict
        return pages_categories_dict

# call it (it took 1 minute on my macbook pro)
%time ef= create_page_category_dict('nlwiki-20150602-pages-articles1.xml.bz2')
```

In [6]:
# http://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup
small= open("NLWikipedia/small.xml")
soup = BeautifulSoup(small)

In [7]:
# list first ten  pages in our small dump
soup.findAll('title')[:10]

[<title>Albert Speer</title>,
 <title>Andre Agassi</title>,
 <title>Groen (partij)</title>,
 <title>Anthony Fokker</title>,
 <title>Albert Plesman</title>,
 <title>Alan Turing</title>,
 <title>Algoritme</title>,
 <title>Alpen</title>,
 <title>Antwerpen (stad)</title>,
 <title>Astronomie</title>]

In [40]:
pages= soup.findAll('page')

page_infobox_dict = {p.title.text:p.find('text') for p in pages}
   
speertext=page_infobox_dict['Albert Speer']

speertext.text[:2000]

u'{{Zie artikel|Zie [[Albert Speer (junior)|Albert Speer jr.]] voor de zoon van Albert Speer, ook een Duits planoloog.}}\n{{Infobox persoon Tweede Wereldoorlog\n| naam=Albert Speer\n| afbeelding=Albert Speer Neurenberg.JPG\n| onderschrift=Speer op het [[Proces van Neurenberg]] (1945-1946)\n| gbdat=19 maart 1905\n| sterfdat=1 september 1981\n| geslacht=Man\n| gbplaats=Mannheim\n| sterfplaats=Londen\n| zijde=Nazi-Duitsland\n| org=NSDAP\n| spfunc=Minister van Bewapening en Munitie\n| rang=\n}}\n\'\'\'Berthold Konrad Hermann Albert Speer\'\'\' ([[Mannheim]], [[19 maart]] [[1905]] \u2013 [[Londen]], [[1 september]] [[1981]]) was een [[Duitsland|Duits]] [[architect]] en tijdens de [[nationaalsocialisme|nazi-heerschappij]] over Duitsland (1933-1945) achtereenvolgens rijksbouwmeester en minister van bewapening. Door zijn vriendschap met de dictator [[Adolf Hitler]] en zijn ministerschap gold hij als een van de machtigste mannen van het [[nazi-Duitsland|Derde Rijk]].<ref>[http://www.conne-islan